<a href="https://colab.research.google.com/github/fgsantosti/programacaodispositivosmoveis/blob/main/Atividade_08_CBF_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade - CBF Stats

Você é analista de desempenho de um clube da Série A e precisa **propor um aplicativo mobile** para análise de dados dos jogadores do Brasileirão 2025, usando:

- **Base de dados**: [Repositório caRtola](https://github.com/henriquepgomide/caRtola)  
- **Referência de jogo fantasy**: [Cartola FC](https://play.google.com/store/apps/details?id=br.com.mobits.cartolafc&hl=pt_BR)  
- **Ecosistema**:
  - Backend (API, linguagem livre)
  - Frontend em **Flutter**
  - Análise de dados com **Pandas**, entregando CSV/JSON para a API


---

### 1. Objetivo do Aplicativo

Criar um **aplicativo mobile** que permita:

1. Visualizar **estatísticas dos jogadores** do Brasileirão (por rodada, por temporada, por posição).
2. Calcular e exibir a **pontuação fantasy** usando os scouts que você listou (defesa + ataque).
3. Ajudar o analista a:
   - Comparar jogadores (do próprio clube vs. liga)
   - Monitorar evolução (últimas N rodadas)
   - Ver impacto de scouts específicos (ex: desarmes, finalizações, cartões, etc.).

---

### 2. Modelo de Pontuação (Regra de Negócio)

Você já trouxe os pesos. Abaixo, organizo em “tabela” para virar fácil regra de cálculo na API/Pandas.

#### 2.1. Scouts de Defesa

- `DS` – Desarmes: **+1,2**
- `FC` – Falta cometida: **-0,3**
- `GC` – Gol contra: **-3,0**
- `CA` – Cartão amarelo: **-1,0**
- `CV` – Cartão vermelho: **-3,0**
- `SG` – Jogo sem sofrer gol  
  - **+5,0** (apenas para **Goleiro, Zagueiro, Lateral**)
- `DE` – Defesa difícil  
  - **+1,0** (apenas **Goleiro**)
- `DP` – Defesa de pênalti  
  - **+7,0** (apenas **Goleiro**)
- `GS` – Gol sofrido  
  - **-1,0** (apenas **Goleiro**)
- `PC` – Pênalti cometido: **-1,0**

#### 2.2. Scouts de Ataque

- `FS` – Falta sofrida: **+0,5**
- `PE` – Passe incompleto: **-0,1**
- `A` – Assistência: **+5,0**
- `FT` – Finalização na trave: **+3,0**
- `FD` – Finalização defendida: **+1,2**
- `FF` – Finalização para fora: **+0,8**
- `G` – Gol: **+8,0**
- `I` – Impedimento: **-0,1**
- `PP` – Pênalti perdido: **-4,0**
- `PS` – Pênalti sofrido: **+1,0**

---

### 3. Arquitetura Geral

#### 3.1. Visão de Alto Nível

1. **Módulo de Análise (Python + Pandas)**  
   - Faz ingestão dos dados do [caRtola](https://github.com/henriquepgomide/caRtola).
   - Normaliza scouts, posições, rodadas, jogadores.
   - Calcula métricas agregadas (médias, soma por temporada, por rodada, etc.).
   - Exporta **CSV/JSON** com:
     - `jogador`, `clube`, `posicao`, `rodada`, `scouts_brutos`, `pontuacao_fantasy`, etc.

2. **Backend (API REST)** – por exemplo:
   - Linguagem sugerida: **Python (FastAPI / Django REST)** ou Node/Express.
   - Carrega os dados processados (CSV/JSON) em memória ou banco.
   - Expõe endpoints:
     - Lista de jogadores, filtros por clube/posição.
     - Estatísticas por jogador/rodada.
     - Comparação entre jogadores.
     - Ranking por rodada/temporada.

3. **Frontend (Flutter)**  
   - Consome os endpoints da API.
   - Telas para:
     - Lista de jogadores
     - Detalhes do jogador
     - Comparações
     - Rankings e dashboards simples (gráficos de linha/barras).

---

### 4. Módulo de Análise de Dados (Pandas)

#### 4.1. Fluxo com Pandas

1. **Carregar dados do repositório caRtola**  
   - Explorar quais CSV/arquivos existem com scouts de jogadores, rodadas, posições.
2. **Limpeza e padronização**:
   - Garantir que cada linha represente: `jogador x partida x rodada`.
   - Ajustar nomes dos scouts para bater com os códigos (`DS`, `FC`, etc.).
3. **Criar função de cálculo de pontuação**:
   - Recebe uma linha (jogador-partida) + posição.
   - Aplica pesos.
4. **Gerar DataFrames prontos para API**:
5. **Exportar**:
   - `jogadores_rodadas.csv` e/ou `jogadores_rodadas.json`.

6. - **Base de para as analises**: [Repositório fgsantosti](https://github.com/fgsantosti/topicos_especiais_desenvolvimento/blob/main/analise_dados_brasileirao.ipynb)  

---

### 5. Backend (API)

Ex.
- **Python + FastAPI**

Pode usar **banco relacional simples** (PostgreSQL, MySQL, SQLite) ou até somente **arquivos CSV/JSON em disco e carregados na memória**, dependendo do escopo da atividade.

#### 5.2. Endpoints Sugeridos

Alguns exemplos REST:

1. `GET /jogadores`
   - Query params: `clube`, `posicao`, `nome` (search)
   - Retorna lista de jogadores filtrada.

2. `GET /jogadores/{id_jogador}`
   - Dados básicos do jogador + resumo de stats da temporada.

3. `GET /jogadores/{id_jogador}/rodadas`
   - Query: `temporada`, `limite` (ex: últimas 5)
   - Retorna lista de jogos (rodada, adversário, scouts, pontos_fantasy).

4. `GET /ranking/rodada`
   - Params: `rodada`, `posicao` opcional
   - Retorna top N jogadores da rodada pela pontuação.

5. `GET /comparacao`
   - Params: `id_jogador1`, `id_jogador2`
   - Retorna stats comparativas (pontos médios, scouts principais, gráfico de evolução).

6. `GET /estatisticas/clube/{clube}`
   - Overview dos jogadores do clube na temporada (útil para o analista do seu clube).


7. Top jogadores por assistências (A)

**Descrição:** Retorna os jogadores com maior número de **assistências (A)** em uma temporada ou rodada.

- **Endpoint:**  
  `GET /scouts/ataque/top-assistencias`
- **Query Params (exemplos):**
  - `temporada=2025`
  - `rodada=3` (opcional – se não passar, considera a temporada inteira)
  - `limite=10`
  - `posicao=MEI` (opcional)
  - `clube=FLA` (opcional)
- **Uso:** “Top 10 jogadores com mais assistências na rodada 3 de 2025”.

8. Top jogadores por desarmes (DS)

**Descrição:** Retorna os jogadores com maior número de **desarmes (DS)**.

- **Endpoint:**  
  `GET /scouts/defesa/top-desarmes`
- **Query Params:**
  - `temporada=2025`
  - `rodada=3` (opcional)
  - `limite=10`
  - `posicao=ZAG` (opcional)
  - `clube=PAL` (opcional)
- **Uso:** “Top 10 zagueiros em desarmes na temporada 2025”.

9. Top jogadores por gols (G)

**Descrição:** Retorna os jogadores com maior número de **gols (G)**.

- **Endpoint:**  
  `GET /scouts/ataque/top-gols`
- **Query Params:**
  - `temporada=2025`
  - `rodada=3` (opcional)
  - `limite=10`
  - `posicao=ATA` (opcional)
  - `clube=` (opcional)
- **Uso:** “Top 10 atacantes em gols na temporada”.

10. Top jogadores por finalizações no alvo (FD + FT)

**Descrição:** Retorna os jogadores com maior número de finalizações que levaram perigo:
- `FD` – Finalização defendida  
- `FT` – Finalização na trave  

Você pode somar `FD + FT` em uma métrica só de “finalizações perigosas”.

- **Endpoint:**  
  `GET /scouts/ataque/top-finalizacoes-perigosas`
- **Query Params:**
  - `temporada=2025`
  - `rodada=3` (opcional)
  - `limite=10`
  - `posicao=` (opcional)
  - `clube=` (opcional)
- **Uso:** “Top 10 jogadores que mais finalizam com perigo ao gol”.

11. Top jogadores por faltas sofridas (FS)

**Descrição:** Retorna os jogadores que mais **sofreram faltas (FS)**.

- **Endpoint:**  
  `GET /scouts/ataque/top-faltas-sofridas`
- **Query Params:**
  - `temporada=2025`
  - `rodada=3` (opcional)
  - `limite=10`
  - `posicao=MEI` (opcional)
  - `clube=` (opcional)
- **Uso:** “Top 10 jogadores mais caçados em campo”.

12. Top jogadores por faltas cometidas (FC)

**Descrição:** Retorna os jogadores que mais **cometeram faltas (FC)**.

- **Endpoint:**  
  `GET /scouts/defesa/top-faltas-cometidas`
- **Query Params:**
  - `temporada=2025`
  - `rodada=3` (opcional)
  - `limite=10`
  - `posicao=ZAG` (opcional)
  - `clube=` (opcional)
- **Uso:** “Top 10 jogadores que mais fizeram faltas na temporada”.

13. Top goleiros em defesas difíceis (DE)

**Descrição:** Exclusivo para goleiros: quem mais fez **defesas difíceis (DE)**.

- **Endpoint:**  
  `GET /scouts/goleiros/top-defesas-dificeis`
- **Query Params:**
  - `temporada=2025`
  - `rodada=3` (opcional)
  - `limite=10`
  - `clube=` (opcional)
- **Regra:** Filtrar apenas `posicao = GOL`.
- **Uso:** “Top 10 goleiros em defesas difíceis”.

14. Top goleiros em pênaltis defendidos (DP)

**Descrição:** Goleiros com mais **defesas de pênalti (DP)**.

- **Endpoint:**  
  `GET /scouts/goleiros/top-penaltis-defendidos`
- **Query Params:**
  - `temporada=2025`
  - `limite=10`
  - `clube=` (opcional)
- **Uso:** “Top goleiros pegadores de pênalti”.

15. Top defensores em jogos sem sofrer gol (SG)

**Descrição:** Para **Goleiros, Zagueiros e Laterais**, quem mais teve **jogos sem sofrer gol (SG)**.

- **Endpoint:**  
  `GET /scouts/defesa/top-jogos-sem-gol`
- **Query Params:**
  - `temporada=2025`
  - `rodada=` (opcional; se vazio, considerar acumulado na temporada)
  - `limite=10`
  - `clube=` (opcional)
- **Regra:** Filtrar posições em `{GOL, ZAG, LAT}`.
- **Uso:** “Top 10 defensores com mais SG na temporada”.



---

### 6. Frontend (Flutter)

#### 6.1. Principais Telas

1. **Tela de Login/Seleção de Clube** (opcional, se quiser restringir a um clube)
2. **Dashboard Inicial**
   - Filtro por:
     - Rodada (atual, última, selecionar outra)
     - Posição
   - Cards:
     - Top 5 jogadores do clube na rodada
     - Top 5 da liga

3. **Lista de Jogadores**
   - Lista com:
     - Nome
     - Clube
     - Posição
     - Média de pontos
     - Pontos na última rodada
   - Barra de busca/filros.

4. **Detalhe do Jogador**
   - Dados básicos:
     - Foto (se disponível), nome, clube, posição.
   - Estatísticas:
     - Média temporada
     - Média últimas 5
     - Pontos máxima/mínima
   - Histórico por rodada (lista ou gráfico).

5. **Tela de Comparação de Jogadores**
   - Seleciona 2 ou mais jogadores.
   - Mostra:
     - Pontos médios
     - Distribuição de scouts (ex: percentual de pontos vindos de desarmes vs gols).
     - Gráfico de linhas rodadas x pontos.

6. **Tela de Rankings**
   - Ranking por rodada.
   - Ranking geral temporada.
   - Ranking somente do seu clube.

#### 6.2. Integração com API

- Utilizar `http` ou `dio` no Flutter.
- Criar serviços como:
  - `PlayerService` com métodos: `getPlayers`, `getPlayerDetails`, `getPlayerRounds`, etc.
- Tratar cache simples (pode ser local storage) para não depender sempre da rede.

---

### 7. Como amarrar tudo na sua entrega da atividade

Para tornar sua atividade bem completa, você pode entregar:

1. **Documento / Relatório** (ou slides) contendo:
   - Contexto do problema (analista de desempenho, Brasileirão 2025, inspiração Cartola).
   - Descrição das regras de pontuação (tabelas de scouts).
   - Diagrama simples da arquitetura:
     - Pandas → CSV/JSON → API → Flutter.
   - Descrição dos principais endpoints da API.
   - Wireframes ou descrição das telas do aplicativo.

2. **Anexos Técnicos (se for pedido)**:
   - Script em Python com:
     - Leitura de dados do repo caRtola.
     - Cálculo de pontuação.
     - Geração de CSV/JSON.
   - Exemplo de definição de endpoint (em FastAPI, por exemplo).
   - Modelo de requisição/resposta JSON que o Flutter consumiria.

---
